In [1]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-rplkjfyx
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-rplkjfyx
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=8dbfe383c87a5797b1cbbbc60d31294dd90a9046bd646349332981dbe85b70d6
  Stored in directory: /tmp/pip-ephem-wheel-cache-ns5n6b50/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper
Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get

In [2]:
!whisper "Arpaaudio.mp4" --model medium

100%|██████████████████████████████████████| 1.42G/1.42G [00:14<00:00, 105MiB/s]
Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English
[00:00.000 --> 00:11.380]  Welcome to English in a Minute. If you've ever gardened, you know weeds grow super fast.
[00:11.380 --> 00:18.760]  They can take over a garden overnight. But what does grow like a weed mean in other situations?
[00:18.760 --> 00:23.280]  Hey, Dan. Do you want to see pictures from my trip home?
[00:23.280 --> 00:27.080]  Oh, geez. I would love to, but...
[00:27.080 --> 00:31.240]  This is me with my little cousin Arlo.
[00:31.240 --> 00:34.480]  Little? He's taller than you.
[00:34.480 --> 00:38.720]  I know. He is growing like a weed. He's only ten.
[00:38.720 --> 00:42.120]  Does he have a beard?
[00:42.120 --> 00:51.440]  To grow like a weed just means to grow very quickly. We mostly use this expression when
[00:51.440 --> 00:54.480]  talking about how fast 

In [7]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install prerequisites
!pip install transformers datasets torch pandas openpyxl rouge-score bert-score fastapi uvicorn pyngrok
!pip install accelerate  # For Trainer

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.0 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=aaa0faae4cf728483064104d31e1f738aa9d5ff5a8c4918d6d69c9445f52eca5
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [8]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer

# Load the Excel file
file_path = '/content/drive/MyDrive/Arpa UIU/SOAP_Assessment_Data/medical_dialogue_test.xlsx'
df = pd.read_excel(file_path)

# Assume columns: 'dialogue' and 'soap'
# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Split: 80% train, 10% validation, 10% test
train_test_split = dataset.train_test_split(test_size=0.2, seed=42)
val_test_split = train_test_split['test'].train_test_split(test_size=0.5, seed=42)
train_dataset = train_test_split['train']
val_dataset = val_test_split['train']
test_dataset = val_test_split['test']

# Tokenizer for BART
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

# Preprocessing function
def preprocess_function(examples):
    inputs = tokenizer(examples["dialogue"], max_length=512, truncation=True, padding="max_length")
    targets = tokenizer(examples["soap"], max_length=150, truncation=True, padding="max_length")
    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": targets["input_ids"],
    }

# Apply preprocessing
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

# Set format for PyTorch
tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_val.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

print("Data prepared. Train size:", len(tokenized_train), "Val size:", len(tokenized_val), "Test size:", len(tokenized_test))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

Data prepared. Train size: 200 Val size: 25 Test size: 25


In [10]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install prerequisites
!pip install transformers datasets torch pandas openpyxl rouge-score bert-score fastapi uvicorn pyngrok peft accelerate bitsandbytes
!pip install sentence-transformers  # For handling embeddings if needed

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 17.8 MB/s eta 0:00:00


In [13]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

# Install all required libraries
!pip install transformers datasets torch pandas openpyxl rouge-score bert-score fastapi uvicorn pyngrok accelerate

# Import necessary modules
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, pipeline
from rouge_score import rouge_scorer
from bert_score import score as bert_score
import numpy as np
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
from pyngrok import ngrok

print("Prerequisites installed and imports completed.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Prerequisites installed and imports completed.


In [14]:
# File paths
train_path = '/content/drive/MyDrive/Arpa UIU/SOAP_Assessment_Data/medical_dialogue_train.csv'
test_path = '/content/drive/MyDrive/Arpa UIU/SOAP_Assessment_Data/medical_dialogue_test.xlsx'
val_path = '/content/drive/MyDrive/Arpa UIU/SOAP_Assessment_Data/medical_dialogue_validation.xlsx'

# Load datasets
train_df = pd.read_csv(train_path)
test_df = pd.read_excel(test_path)
val_df = pd.read_excel(val_path)

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
val_dataset = Dataset.from_pandas(val_df)

# Tokenizer for BART (handles Bengali via multilingual support)
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

# Preprocessing function
def preprocess_function(examples):
    inputs = tokenizer(examples["dialogue"], max_length=512, truncation=True, padding="max_length")
    targets = tokenizer(examples["soap"], max_length=150, truncation=True, padding="max_length")
    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": targets["input_ids"],
    }

# Apply preprocessing
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)

# Set format for PyTorch
tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_val.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

print(f"Data prepared. Train size: {len(tokenized_train)}, Val size: {len(tokenized_val)}, Test size: {len(tokenized_test)}")

Map:   0%|          | 0/9250 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Data prepared. Train size: 9250, Val size: 500, Test size: 250


In [22]:
# Load pre-trained BART model
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")

# Training arguments (optimized for accuracy, coherence, and conciseness)
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Fixed: Changed from 'evaluation_strategy' to 'eval_strategy'
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,  # Adjust if GPU memory is low
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    logging_dir="./logs",
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained("./fine_tuned_bart")
tokenizer.save_pretrained("./fine_tuned_bart")

print("Model fine-tuned and saved to './fine_tuned_bart'.")

/tmp/ipython-input-3944804613.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: arpasaha-cse (arpasaha-cse-united-international-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,1.007200,0.824692
2,0.831400,0.768471
3,0.726400,0.739355


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


Model fine-tuned and saved to './fine_tuned_bart'.


In [29]:
# Load fine-tuned model for inference
summarizer = pipeline("summarization", model="./fine_tuned_bart", tokenizer="./fine_tuned_bart")

# Function to generate summaries
def generate_summary(dialogue):
    return summarizer(dialogue, max_length=150, min_length=50, do_sample=False)[0]["summary_text"]

# Baseline: Use pre-trained BART without fine-tuning
baseline_summarizer = pipeline("summarization", model="facebook/bart-base")

# Evaluate on test set (use original test_dataset to access "dialogue" and "soap" columns)
references = [example["soap"] for example in test_dataset]  # Fixed: Use test_dataset, not tokenized_test
predictions = [generate_summary(example["dialogue"]) for example in test_dataset]  # Fixed: Use test_dataset
baseline_predictions = [baseline_summarizer(example["dialogue"], max_length=150, min_length=50, do_sample=False)[0]["summary_text"] for example in test_dataset]  # Fixed: Use test_dataset

# ROUGE scores for fine-tuned model
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = [scorer.score(pred, ref) for pred, ref in zip(predictions, references)]
avg_rouge1 = np.mean([s['rouge1'].fmeasure for s in rouge_scores])
avg_rouge2 = np.mean([s['rouge2'].fmeasure for s in rouge_scores])
avg_rougeL = np.mean([s['rougeL'].fmeasure for s in rouge_scores])

# BERTScore for fine-tuned model (using English; switch to 'bn' if Bengali model available)
P, R, F1 = bert_score(predictions, references, lang="en", verbose=False)
avg_bert_f1 = F1.mean().item()

# Baseline ROUGE for comparison
baseline_rouge_scores = [scorer.score(pred, ref) for pred, ref in zip(baseline_predictions, references)]
baseline_avg_rouge1 = np.mean([s['rouge1'].fmeasure for s in baseline_rouge_scores])

# Print results
print("=== Evaluation Results ===")
print(f"Fine-Tuned Model - ROUGE-1: {avg_rouge1:.2f}, ROUGE-2: {avg_rouge2:.2f}, ROUGE-L: {avg_rougeL:.2f}, BERTScore F1: {avg_bert_f1:.2f}")
print(f"Baseline Model - ROUGE-1: {baseline_avg_rouge1:.2f}")

# Qualitative Analysis: Examples (use test_dataset for original columns)
print("\n=== Qualitative Analysis ===")
print("Baseline Example (Pre-trained BART):")
print(f"Input: {test_dataset[0]['dialogue']}")  # Fixed: Use test_dataset
print(f"Baseline Summary: {baseline_predictions[0]}")
print(f"Reference: {references[0]}")
print("Analysis: Often verbose and lacks SOAP structure.")

print("\nGood Fine-Tuned Example:")
print(f"Input: {test_dataset[0]['dialogue']}")  # Fixed: Use test_dataset
print(f"Generated: {predictions[0]}")
print(f"Reference: {references[0]}")
print("Analysis: Accurate, coherent, concise, and SOAP-structured.")

print("\nBad Fine-Tuned Example:")
print(f"Input: {test_dataset[1]['dialogue']}")  # Fixed: Use test_dataset
print(f"Generated: {predictions[1]}")
print(f"Reference: {references[1]}")
print("Analysis: May miss details or hallucinate due to limited training data.")

# Save detailed report
with open("evaluation_report.txt", "w") as f:
    f.write("=== SOAP Summarization Evaluation Report ===\n")
    f.write(f"Fine-Tuned Model Metrics:\nROUGE-1: {avg_rouge1:.2f}\nROUGE-2: {avg_rouge2:.2f}\nROUGE-L: {avg_rougeL:.2f}\nBERTScore F1: {avg_bert_f1:.2f}\n")
    f.write(f"Baseline ROUGE-1: {baseline_avg_rouge1:.2f}\n\nQualitative Examples:\n" + "\n".join([
        f"Baseline - Input: {test_dataset[0]['dialogue']}\nGenerated: {baseline_predictions[0]}\nReference: {references[0]}\n",  # Fixed: Use test_dataset
        f"Good - Input: {test_dataset[0]['dialogue']}\nGenerated: {predictions[0]}\nReference: {references[0]}\n",  # Fixed: Use test_dataset
        f"Bad - Input: {test_dataset[1]['dialogue']}\nGenerated: {predictions[1]}\nReference: {references[1]}\n"  # Fixed: Use test_dataset
    ]))

print("Report saved to 'evaluation_report.txt'.")

Device set to use cuda:0
Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/t

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


=== Evaluation Results ===
Fine-Tuned Model - ROUGE-1: 0.47, ROUGE-2: 0.31, ROUGE-L: 0.36, BERTScore F1: 0.89
Baseline Model - ROUGE-1: 0.35

=== Qualitative Analysis ===
Baseline Example (Pre-trained BART):
Input: Doctor: Hello, can you please tell me about your past medical history?
Patient: Hi, I don't have any past medical history.
Doctor: Okay. What brings you in today?
Patient: I've been experiencing painless blurry vision in my right eye for a week now. I've also had intermittent fevers, headache, body aches, and a nonpruritic maculopapular rash on my lower legs for the past 6 months.
Doctor: Thank you for sharing that. Have you had any other symptoms such as neck stiffness, nausea, vomiting, Raynaud's phenomenon, oral ulcerations, chest pain, shortness of breath, abdominal pain, or photosensitivity?
Patient: No, only an isolated episode of left knee swelling and testicular swelling in the past.
Doctor: Do you work with any toxic substances or have any habits like smoking, drink

In [36]:
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import pipeline

# Define FastAPI app
app = FastAPI(title="SOAP Medical Chat Summarizer", description="API for generating SOAP summaries from medical dialogues.")

class SummarizeRequest(BaseModel):
    dialogue: str

# Load the fine-tuned model (ensure ./fine_tuned_bart is uploaded to the Space)
summarizer = pipeline("summarization", model="./fine_tuned_bart", tokenizer="./fine_tuned_bart")

@app.post("/summarize")
def summarize(request: SummarizeRequest):
    summary = summarizer(request.dialogue, max_length=150, min_length=50, do_sample=False)[0]["summary_text"]
    return {"summary": summary}

Device set to use cuda:0
